# Pinyin Syllable Count Generator Working Draft Notes
This document details the process of experimentation related to the construction of the Pinyin Syllable Count Generator module. The module takes two columns of string data, one with unboken text in Chinese, the other with its pinyin counterpart. The package will evaluate the number of syllables in the chinese text based on the number of syllables it can detect in the supplied Latin characters. The results will be delivered as integers with commas separating the syllables in each individual word.

The Pandas package will be essential for the vector manipulation and storage.

In [3]:
import pandas as pd
import numpy as np
import csv
import sys

Because pinyin is a structured system with accepted combinations of "initials" (the initial sound of a syllable, e.g. the first consonant) and "finals" (the ending sound of a syllable, e.g. vowels ending with a consonant). One syllable in pinyin can be as small as one letter and as large as six letters. Therefore, detecting a syllable can occur in one of two ways:
1. A sequence of pinyin letters can be compared to a list of accepted pinyin syllables. If the sequence appears in the list, or if the sequence is part of a longer accepted term ("tia" by itself is not an accepted syllable, however "tiao" and "tian" are), then an additional letter from the evaluated word can be added to the sequence and reevaluated. If the sequence does not appear in the list and is not part of a longer accepted term, then the last character of the sequence becomes the first character of the next syllable and the syllable count is incremented.
2. A sequence of pinyin letters is evaluated against a DataFrame of initials and finals, utilizing the .loc command to spot matches if they exist or null values if they don't.

Given these possiblities two metrics need to be considered in order to settle on the proper method for the evaluation function: speed and size. Comparing a sequence to a list of terms can be faster, however the list itself may be excessively large. Evaluating sequences by their combination of initials and finals, however, may take longer but will be result in a much smaller package. The following exercises before will test each method in a smaller scale to get a general sense of the speed vs. size issue.

## Setup for small tests

An example sequence of letters will be tested with both methods.

In [4]:
## LIST ##
# Import list of all pinyin combinations
smList = []
with open("pinyinList.csv") as f:
    r = csv.reader(f)
    for row in r:
        smList += row
        
## NUMPY/DATAFRAME ##
# Variables for evaluation
smInitial = 't'
smFinal = 'eng'

# List of initials, finals for Numpy evaluation
initList = ['ø','b','p','m','f','d','t','n','l','z','c','s',
            'zh','ch','sh','r','j','q','x','g','k','h','y','w']
finList = ['a','ai','ao','an','ang','e','ei','en','eng','er',
           'o','ong','ou','i','ia','iao','ie','iu','ian','iang',
           'in','ing','iong','u','ua','uo','ue','ui','uai','uan','uen','uang','un','v','ve']

# Import Numpy array from CSV
smAr = np.genfromtxt('pinyinDF.csv', delimiter=',', skip_header=1, filling_values=0
                    ).astype('bool_')
smAr = np.delete(smAr, 0, 1)

# Import dataframe from CSV 
smDF = pd.read_csv('pinyinDF.csv', header=0, index_col=0, dtype={'INDEX':str})
smDF = smDF.fillna(0).astype('bool')

print('Python List:')
print(str(smList[50:55]) + '\n')
print('Numpy Array:')
print(str(smAr[0])  + '\n')
print('Pandas DataFrame:')
print(smDF.head(3))

Python List:
['bu', 'pa', 'pai', 'pao', 'pan']

Numpy Array:
[ True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True  True  True  True  True  True  True False
 False False False False False False False False False False False]

Pandas DataFrame:
      a    ai    ao    an   ang      e    ei    en   eng     er  ...     uo  \
ø  True  True  True  True  True   True  True  True  True   True  ...  False   
b  True  True  True  True  True  False  True  True  True  False  ...  False   
p  True  True  True  True  True  False  True  True  True  False  ...  False   

      ue     ui    uai    uan    uen   uang     un      v     ve  
ø  False  False  False  False  False  False  False  False  False  
b  False  False  False  False  False  False  False  False  False  
p  False  False  False  False  False  False  False  False  False  

[3 rows x 35 columns]


In [3]:
# Method 1: Python List
print('Python List: ' + str(sys.getsizeof(smList)))
%timeit -n 100000 smEval in smList

# Method 2: Numpy Array
print('\nNumpy array: ' + str(sys.getsizeof(initList) +
                              sys.getsizeof(finList) +
                              sys.getsizeof(smAr)) +
      ', without initials and finals lists: ' + str(sys.getsizeof(smAr)))
%timeit -n 100000 smAr[initList.index(smInitial),finList.index(smFinal)] != True

# Method 3: Pandas DataFrame
# 3a: slice
print('\nDataFrame: ' + str(sys.getsizeof(smDF)))
print('slice')
%timeit -n 100000 smDF[smFinal][smInitial] != True

# 3b: loc
print('loc')
%timeit -n 100000 smDF.loc[smInitial,smFinal] != True

print('iloc')
# 3c: iloc
%timeit -n 100000 smDF.iloc[initList.index(smInitial),finList.index(smFinal)] != True

Python List: 3760
1.32 µs ± 8.31 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Numpy array: 1552, without initials and finals lists: 952
1.48 µs ± 37.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

DataFrame: 2443
slice
9.05 µs ± 59.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
loc
7.71 µs ± 43.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
iloc
9.08 µs ± 51.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [61]:
def sylCount(pinyin, test=False):
    '''Count the number of Chinese characters based on inputted pinyin text.'''

    # Lists for reference
    consonant = ['b','c','ch','d','f','g','h','j','k','l','m','n',
                 'p','q','r','s','sh','t','w','x','y','z','zh']
    vowel = ['a','e','i','o','u','v']
    
    # Variables used for pinyin evaluation
    
    pinyin = pinyin.strip()

    initial = ''
    final = ''
    newSyl = False
    syl = 1
    sylCount = []
    er = False
    
    
    for i,n in enumerate(str.lower(pinyin)):
        
        endOfWord = i == len(pinyin) - 1 or pinyin[i+1] == ' '
        
        if n == ' ':
            if pinyin[i+1] == ' ':
                if test==True: print('0. space - discard extra space')
                continue
            else:
                sylCount.append(syl)
                newSyl = True
                if test==True: print('0. ' + initial + '|' + final + ' = space - new word')

        elif final:
            if final[-1] == 'e' and n == 'r':
                if not endOfWord:
                    newSyl = True
                    er = True
                    if test==True: print('1. ' + initial + '|' + final + n + ' = er - newSyl')
                else:
                    if len(final) > 1:
                        syl += 1
                        if test==True: print('1. ' + initial + '|' + final + n + ' er - add syl')
                        if test==True: print('new syl: ' + str(syl))
                
            else:
                try:
                    result = smAr[initList.index(initial),finList.index(final + n)]
                    if test==True: print('1a. ' + initial + '|' + final + n + ' = init - trying: ' + str(result))
                except:
                    result = False
                    if test==True: print('1b. ' + initial + '|' + final + n + ' = fin - error')
            
                if not result and not any(i for i in finList if (final+n) in i):
                    newSyl = True
                    if test==True: print('2a. ' + initial + '|' + final + n + ' = fin - new syl')
                else:
                    final += n
                    if test==True: print('2b. ' + initial + '|' + final + ' = fin - maybe more')

        else:
            if n not in vowel:
                initial += n
                if test==True: print('3. ' + initial + ' = init - cons')
            elif not initial:
                initial = 'ø'
                final = n
                if test==True: print('4. ' + initial + '|' + final + ' = init - vowel')
            else:
                final = n
                if test==True: print('5. ' + initial + '|' + final + ' = fin - vowel')
                
        if newSyl:
            if n in vowel:
                initial = 'ø'
                final = n
                syl += 1
            elif n != ' ':
                if er and pinyin[i+1] not in vowel:
                    initial = ''
                    if len(final) > 1:
                        syl+= 1
                else:
                    initial = n
                final = ''
                syl += 1
            else:
                initial = ''
                final = ''
                syl = 1
            if test==True: print('new syl: ' + str(syl))
            newSyl = False
    
    sylCount.append(syl)
    
    return sylCount

In [62]:
print(sylCount('A Honga', True))

4. ø|a = init - vowel
0. ø|a = space - new word
new syl: 1
3. h = init - cons
5. h|o = fin - vowel
1b. h|on = fin - error
2b. h|on = fin - maybe more
1a. h|ong = init - trying: True
2b. h|ong = fin - maybe more
1b. h|onga = fin - error
2a. h|onga = fin - new syl
new syl: 2
[1, 2]


In [63]:
names = pd.read_csv('Chinese Names.csv')
names[names['ChiName'].str.len() > 5]

,EngName,ChiName
102,A Shinaaboshe,阿史那阿波設
104,A Shinachuoluokehan,阿史那啜羅可汗
106,A Shinahuseluo,阿史那斛瑟羅
107,A Shinaqiminkehan,阿史那啟民可汗
120,Abilule Suofeiyinga,阿畢魯勒索費英阿
123,Aboerjijite Aleqinga,阿博爾濟吉特阿勒清阿
152,Ademishimenggu,阿的彌失蒙古
153,Ademishitiemuer,阿的迷失帖木兒
156,Adouhalamanhudoupuhuayaozhu,阿都哈剌蠻護都普花咬住
170,Aerhaihesaer,阿兒孩合撒兒


In [64]:
names['syl'] = names['EngName'].apply(sylCount, test=False)
names['check'] = names['ChiName'].str.len() == names['syl'].apply(sum)
names[~names['check']]

,EngName,ChiName,syl,check
198,Ai Airen,愛仁,"[1, 2]",False
199,Ai Aixinga,愛興阿,"[1, 3]",False
433,Aixinjueluo Bukuliyongshun,布庫里雍順,"[4, 5]",False
455,Aixinjueluo Chongshan,充善,"[4, 2]",False
500,Aixinjueluo Duoergun,多尔衮,"[4, 3]",False
504,Aixinjueluo Eerdeng,愛新覺羅爾登,"[4, 3]",False
534,Aixinjueluo Fulin,福临,"[4, 2]",False
535,Aixinjueluo Fuman,福滿,"[4, 2]",False
592,Aixinjueluo Hongli,弘历,"[4, 2]",False
615,Aixinjueluo Huangtaiji,皇太极,"[4, 3]",False


In [76]:
names.iloc[64657]

EngName    Hu Mi　
ChiName       胡謐　
syl        [1, 1]
check       False
Name: 64657, dtype: object

In [15]:
names[names['EngName'].str.contains('er')][names['check'] == False]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,EngName,ChiName,syl,check
123,Aboerjijite Aleqinga,阿博爾濟吉特阿勒清阿,[4],False
168,Aerbu Wenxiang,阿爾佈文祥,[2],False
397,Aixinjueluo Baerdai,愛新覺羅巴爾岱,[3],False
398,Aixinjueluo Baerkan,愛新覺羅巴爾堪,[3],False
399,Aixinjueluo Baershan,愛新覺羅巴爾善,[3],False
400,Aixinjueluo Baertu,愛新覺羅巴爾圖,[3],False
406,Aixinjueluo Banbuershan,愛新覺羅班布爾善,[4],False
422,Aixinjueluo Biertena,愛新覺羅弼爾特訥,[4],False
427,Aixinjueluo Boerdou,愛新覺羅博爾都,[3],False
428,Aixinjueluo Boerhe,愛新覺羅博爾和,[3],False


In [34]:
names[names['EngName'].str.contains('or')]

,EngName,ChiName
207,Ai Borun,艾伯閏
412,Aixinjueluo Baorui,愛新覺羅保瑞
560,Aixinjueluo Guorui,愛新覺羅國瑞
1812,Ao Boren,敖伯仁
2637,Bai Xiaorong,白小榮
3035,Bao Baorui,保瑞
3068,Bao Chaoru,鮑朝孺
4099,Bi Boren,畢伯仁
5209,Bore,波惹
5210,Boruotieshuier,波若帖水兒


In [60]:
final = 'eyadh'
len(final)
final[len(final)-1]

'h'